<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/7_3_bond_convexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7.3　債券コンベクシティの算出

　債券利回り価格曲線には、文字通り、デュレーションの直線に対し、大なり小なり、緩やかな凸状のカーブが存在することがわかります。債券投資分析ではデュレーションに加えて、コンベクシティを計測することで、利回り変化がもたらす価格変動の正確な計測を試みています。本例も数理ファイナンス実務のために開発されたオープンソース・ソフトウェア・ライブラリのQuantLibを用いて実装します。前節同様、このQuantLibソフトウェアをGoogle Colab上で、ひとつのコードセルのみを用いてインストールします。

In [ ]:
pip install QuantLib

　ライブラリはQuantLibとnumpyの2つをインポートします。

In [10]:
#[1]ライブラリの読み込み
import QuantLib as ql#数理ファイナンス・ライブラリのインポート
import numpy as np#数値演算ライブラリのインポート

　本例においても、令和6年度国債82回債（償還期間30年）を参考に、30年債の3条件(クーポンレート、最終利回り、額面価格）を設定し、償還期間の違いによるコンベクシティを計算します。

In [17]:
#[2]償還期間・クーポーン・利回りの設定
c = 0.018#クーポン・レート
r = 0.02161#複利最終利回り
N = 100#額面価格

　次に発行日と償還期間および債券発行スケジュールの詳細を設定します。本例では令和6年に発行された30年国債(第82回債）を用いています。各発行債券ごとに、発行日、償還日を入力してください。国・地域により祝日、非営業日が異なるため、米国債や英国債のコンベクシティを算出する場合は、4行目を
https://quantlib-python-docs.readthedocs.io/en/latest/instruments/bonds.html
に基づき変更すること。

In [12]:
#[3]発行日・償還期間・スケジュールの設定
schedule = ql.Schedule(
    ql.Date(16, 6, 2024),       #発行日
    ql.Date(20, 3, 2054),       #償還日
    ql.Period(0, ql.Months),    #間隔
    ql.Japan(),                 #祝休日等の国別カレンダー指定
    ql.Unadjusted,              #非営業日調整
    ql.Unadjusted,              #終了日の非営業日調整
    ql.DateGeneration.Backward, #日付生成のルール
    False,                      #開始日が月末の場合に終了日以外の利払日を月末とするか否か
)

　続いて[4]ではbond=ql.FixedRateBond（）とrate=ql.InterestRate()により、債券と利回りの条件を設定します。その後、3行目にcvx=ql.BondFunctions.convexity()を記述し、1行目と2行目の債券と利回りの変数名を入力することで、コンベキシティを計算します。

In [13]:
#[4]コンベキシティの算出：債券の条件設定
bond = ql.FixedRateBond(
    2,                  #決済に要する日数
    N,                  #額面
    schedule,           #利払日
    [c],                #クーポンレート
    ql.Actual365Fixed() #日数計算方法
)

In [18]:
#[5]コンベキシティの算出：利回りの条件設定
rate = ql.InterestRate(
    r,                  #最終利回り
    ql.Actual360(),     #営業日数計算方法
    ql.Compounded,      #複利か否か
    ql.Annual           #年間利回りか否か
)

In [21]:
#[6]コンベキシティの算出：実装
cvx = ql.BondFunctions.convexity(bond, rate)#コンベクシティの算出

　最後に30年債(第82回債)のコンベクシティの算出結果を可視化するためのソースコードを記述します。

In [22]:
#[7]コンベクシティ算出結果の表示
print('convex:{}'.format(np.round(cvx,decimals=3)))#算出結果の表示

convex:902.473
